In [1]:
#imports
import os
from accelerate import Accelerator, notebook_launcher
import torch
#from parallelformers import parallelize

from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling, OPTForCausalLM, Trainer, TrainingArguments
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset
from utils.save_utils import load_masked_model, load_masked_model_single
from utils.prehook_utils import remove_all_hooks

In [2]:
def training_step():
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    #accelerate = Accelerator()
    
    def encode_tok(examples):
        return tokenizer(examples['text'], truncation=True, padding='max_length')

    #print(torch.cuda.is_initialized())
    model_name='opt-1.3b'
    EPOCH_COUNT=10
    SPARSITY=0.3
    tokenizer = AutoTokenizer.from_pretrained(f'facebook/{model_name}', padding_side='left', model_max_length=1024)
    #print(torch.cuda.is_initialized())
    #stream c4, training split
    training_data = load_dataset('c4', 'en', split='train', streaming=True)
    #training_data = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train', streaming=True)
    #IMPORTANT: process data while streaming -> remove unnecessary columns in batches
    training_data = training_data.map(encode_tok, 
                                      batched=True,
                                      remove_columns=["text", "timestamp", "url"])
    #set data to tensor mode
    training_data = training_data.with_format("torch")

    #dataloader from dataloader (mlm=False when training without mask)
    dataloader = DataLoader(training_data, 
                            collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False),
                            batch_size=1, num_workers=16, pin_memory=True)
    #print(torch.cuda.is_initialized())
    loaded_model = OPTForCausalLM.from_pretrained(f'facebook/{model_name}',
                                                      output_attentions=True,
                                                      output_hidden_states=True).to(device=device)
    #print(torch.cuda.is_initialized())
    if SPARSITY != 1:
        load_masked_model_single(loaded_model, f'pruned_models/{model_name}-{SPARSITY}.pt')
    loaded_model = torch.nn.DataParallel(loaded_model, device_ids=[0,1,2,3])
    #loaded_model = parallelize(loaded_model, num_gpus=4, fp16=True, verbose='detail')
    loaded_model.train()
    
    FROZEN_LIMIT = 18
    for n, p in loaded_model.named_parameters():
        split_name = n.split('layers.')
        if len(split_name) > 1 and int(split_name[-1].split('.')[0]) < FROZEN_LIMIT:
            if p.requires_grad:
                p.requires_grad = False
    #print(torch.cuda.is_initialized())
    t_optim = torch.optim.AdamW(params=loaded_model.parameters(), lr=1e-5)


    #loaded_model, t_optim, dataloader = accelerate.prepare(loaded_model, t_optim, dataloader)
    !nvidia-smi
    t_optim = torch.optim.AdamW(params=loaded_model.parameters(), lr=1e-5)
    #print(torch.cuda.is_initialized())
    for epoch in tqdm(range(EPOCH_COUNT)):
        training_data.set_epoch(epoch)
        for i, batch in enumerate(dataloader):
            #print('TRAINING')
            !nvidia-smi
            #print(batch)
            if i == 5:
                break
            #batch = {k: torch.tensor(v) for k, v in batch.items()}
            #print(batch)
            #print(batch['input_ids'].size())
            #print(batch['attention_mask '].size())
            outputs = loaded_model(**batch)
            loss = outputs.loss
            print(loss)
            #accelerate.backward(loss)
            loss.backward()
            t_optim.step()
            t_optim.zero_grad()
    torch.save(loaded_model.state_dict(), f'pruned_models/{model_name}-{SPARSITY}-finetuned.pt')

In [3]:
training_step()

/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Sun Feb 26 17:22:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   30C    P0    56W / 250W |  17333MiB / 40960MiB |     42%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   

  0%|          | 0/10 [00:00<?, ?it/s]

Sun Feb 26 17:24:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   29C    P0    35W / 250W |  17333MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   

/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


tensor([6.5384], device='cuda:0', grad_fn=<GatherBackward>)
Sun Feb 26 17:24:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    54W / 250W |  38161MiB / 40960MiB |    100%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  O

  0%|          | 0/10 [01:16<?, ?it/s]


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /tmp/ipykernel_13331/2753431802.py:1 in <cell line: 1>                                    │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_13331/2753431802.py'                 │
│                                                                                           │
│ /tmp/ipykernel_13331/1029798185.py:65 in training_step                                    │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_13331/1029798185.py'                 │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1194  │
│ in _call_impl                                                                             │
│                                                                                           │
│   1191 │   │   # this function, and just call forward.                                    │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1195 │   │   # Do not call functions when jit is used                                   │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/data_parallel. │
│ py:171 in forward                                                                         │
│                                                                                           │
│   168 │   │   │   if len(self.device_ids) == 1:                                           │
│   169 │   │   │   │   return self.module(*inputs[0], **kwargs[0])                         │
│   170 │   │   │   replicas = self.replicate(self.module, self.device_ids[:len(inputs)])   │
│ ❱ 171 │   │   │   outputs = self.parallel_apply(replicas, inputs, kwargs)                 │
│   172 │   │   │   return self.gather(outputs, self.output_device)                         │
│   173 │                                                                                   │
│   174 │   def replicate(self, module, device_ids):                                        │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/data_parallel. │
│ py:181 in parallel_apply                                                                  │
│                                                                                           │
│   178 │   │   return scatter_kwargs(inputs, kwargs, device_ids, dim=self.dim)             │
│   179 │                                                                                   │
│   180 │   def parallel_apply(self, replicas, inputs, kwargs):                             │
│ ❱ 181 │   │   return parallel_apply(replicas, inputs, kwargs, self.device_ids[:len(replic │
│   182 │                                                                                   │
│   183 │   def gather(self, outputs, output_device):                                       │
│   184 │   │   return gather(outputs, output_device, dim=self.dim)                         │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-pa

In [3]:
notebook_launcher(training_step, args=(), num_processes=4, mixed_precision='bf16')

Launching training on 4 GPUs.


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /tmp/ipykernel_2469/2085411004.py:1 in <cell line: 1>                                     │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2469/2085411004.py'                  │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/accelerate/launchers.py:136 in   │
│ notebook_launcher                                                                         │
│                                                                                           │
│   133 │   │   │   │   launcher = PrepareForLaunch(function, distributed_type="MULTI_GPU") │
│   134 │   │   │   │                                                                       │
│   135 │   │   │   │   print(f"Launching training on {num_processes} GPUs.")               │
│ ❱ 136 │   │   │   │   start_processes(launcher, args=args, nprocs=num_processes, start_me │
│   137 │   │                                                                               │
│   138 │   │   else:                                                                       │
│   139 │   │   │   # No need for a distributed launch otherwise as it's either CPU, GPU or │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/multiprocessing/spawn.py:1 │
│ 98 in start_processes                                                                     │
│                                                                                           │
│   195 │   │   return context                                                              │
│   196 │                                                                                   │
│   197 │   # Loop on join until it returns True or raises an exception.                    │
│ ❱ 198 │   while not context.join():                                                       │
│   199 │   │   pass                                                                        │
│   200                                                                                     │
│   201                                                                                     │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/multiprocessing/spawn.py:1 │
│ 60 in join                                                                                │
│                                                                                           │
│   157 │   │   original_trace = self.error_queues[error_index].get()                       │
│   158 │   │   msg = "\n\n-- Process %d terminated with the following error:\n" % error_in │
│   159 │   │   msg += original_trace                                                       │
│ ❱ 160 │   │   raise ProcessRaisedException(msg, error_index, failed_process.pid)          │
│   161                                                                                     │
│   162                                                                                     │
│   163 class SpawnContext(ProcessContext):                                                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ProcessRaisedException: 

-- Process 2 terminated with the following error:
Traceback (most recent call last):
  File 
"/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", 
line 69, in _wrap
    fn(i, *args)
  File "/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/accelerate/utils/launch.py",
line 97, in __call__
    self.launcher(*args)
  File "/tmp/ipykernel_2469/1103207772.py"